In [1]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
import random
from random import randint
from pyspark.sql import functions as f
from pyspark.sql.types import StringType


In [2]:
spark_conf = (
SparkConf()
.set("spark.jars.packages", 'org.apache.hadoop:hadoop-client:3.3.0,org.apache.hadoop:hadoop-aws:3.3.0')
.set("spark.driver.memory", "6g")
.set("spark.hadoop.fs.s3a.endpoint", "minio:9000")
.set("spark.hadoop.fs.s3a.access.key", "Hd8vo0i3RJIq7vmIwz6T")
.set("spark.hadoop.fs.s3a.secret.key","gUv5gRpOKvZOxnw6tcApgXcmwvmoZwspA8wGoj8m" )
.set("spark.hadoop.fs.s3a.path.style.access", "true")
.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
)
sc = SparkContext.getOrCreate(spark_conf)
spark = SparkSession(sc)

In [3]:
 print(f"Hadoop version = {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Hadoop version = 3.3.2


In [4]:
N=10000
M=12500
K=150

documentCurrency = ["EUR", "USD", "JPY", "GBR", "CHF"]
documentService = ["Tax", "Rent", "Loan", "Insurance", "Utility"]

from random import randint
from datetime import datetime,timedelta
def invoiceGenerator(k=5):
  postingDate = datetime(2022,1,1)+timedelta(randint(0,200))
  return {
          "customerId":"Customer_{customerId}".format(customerId=str(randint(0,k)+1).zfill(3)),
          "amount":randint(50,10000),
          "documentCurrency":random.choice(documentCurrency), 
          "postingDate":postingDate.strftime("%Y-%m-%d"),
          "dueDate":(postingDate+timedelta(randint(0,60))).strftime("%Y-%m-%d"),
          "fiscalYear":postingDate.strftime("%Y"),
          "documentService":random.choice(documentService)
         }

def invoiceList(k=5,n=1000):
  rawInvoiceList = [invoiceGenerator(k) for k in range(n)]
  rawInvoiceList.sort(key=lambda row: row.get("postingDate"))
  for pos,val in enumerate(rawInvoiceList):
    val["documentNumber"]="2022-{docNum}".format(docNum=str(pos).zfill(5))
  return rawInvoiceList
  
myInvoiceList = invoiceList(K,N)

In [5]:
def paymentGenerator(paymentList):
  documentNumber = "2022-{docNum}".format(docNum=str(randint(0,len(paymentList)-1)).zfill(5))
  payment = [k for k in paymentList if k.get("documentNumber")==documentNumber][0]
  postingDate = datetime.strptime(payment.get("postingDate"),"%Y-%m-%d")
  return { 
          "documentNumber":documentNumber,
          "paymentDate":(postingDate+timedelta(randint(15,90))).strftime("%Y-%m-%d"),
          "valuePaid":randint(1,payment.get("amount")),
          "documentCurrency":payment.get("documentCurrency")
         }


def paymentList(paymentList,m=250):
  return [paymentGenerator(paymentList) for k in range(m)]
   
myPaymentList = paymentList(myInvoiceList,M)  

In [6]:
invoice_rdd = sc.parallelize(myInvoiceList)
invoice_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [7]:
payment_rdd = sc.parallelize(myPaymentList)
payment_rdd

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:274

In [8]:
columns = ['value', 'customerId', 'documentCurrency', 'documentNumber', 'documentService', 'dueDate', 'fiscalYear', 'postingDate']
invoice_df = invoice_rdd.toDF(columns)
invoice_df.printSchema()

root
 |-- value: long (nullable = true)
 |-- customerId: string (nullable = true)
 |-- documentCurrency: string (nullable = true)
 |-- documentNumber: string (nullable = true)
 |-- documentService: string (nullable = true)
 |-- dueDate: string (nullable = true)
 |-- fiscalYear: string (nullable = true)
 |-- postingDate: string (nullable = true)



In [9]:
invoice_df.first()

Row(value=3731, customerId='Customer_368', documentCurrency='EUR', documentNumber='2022-00000', documentService='Loan', dueDate='2022-02-02', fiscalYear='2022', postingDate='2022-01-01')

In [10]:
payment_df = payment_rdd.toDF(["documentCurrency","documentNumber","paymentDate", "valuePaid"])
payment_df.printSchema()

root
 |-- documentCurrency: string (nullable = true)
 |-- documentNumber: string (nullable = true)
 |-- paymentDate: string (nullable = true)
 |-- valuePaid: long (nullable = true)



In [ ]:
absolute_path = "/tmp/my_parquet_data"
invoice_df.write.format('parquet').mode('overwrite').save('')

invoice_df.write.format('parquet').mode('overwrite').save('s3a://spark-demo/users')

Py4JJavaError: An error occurred while calling o105.save.
: java.lang.NoSuchMethodError: 'void com.google.common.base.Preconditions.checkArgument(boolean, java.lang.String, java.lang.Object, java.lang.Object)'
	at org.apache.hadoop.fs.s3a.S3AUtils.lookupPassword(S3AUtils.java:893)
	at org.apache.hadoop.fs.s3a.S3AUtils.lookupPassword(S3AUtils.java:869)
	at org.apache.hadoop.fs.s3a.S3AUtils.getEncryptionAlgorithm(S3AUtils.java:1580)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:341)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileForm

In [12]:
payment_df.first()

Row(documentCurrency='CHF', documentNumber='2022-06012', paymentDate='2022-05-19', valuePaid=2169)

In [15]:
# REPORT WHICH SHOWS THE UNPAID INVOICES

from pyspark.sql import functions as F

payment_sum_df = payment_df.groupBy("documentNumber").agg(F.sum("valuePaid").alias("totalPaid"))

joined_df = invoice_df.join(payment_sum_df, on="documentNumber", how="left")

unpaid_invoices_df = joined_df.filter(joined_df["value"] > F.coalesce(joined_df["totalPaid"], F.lit(0)))

result_df = unpaid_invoices_df.select("value", "customerId", "documentCurrency", "documentNumber", "documentService", "dueDate", "fiscalYear", "postingDate", "totalPaid")

result_df.show()

+-----+-------------+----------------+--------------+---------------+----------+----------+-----------+---------+
|value|   customerId|documentCurrency|documentNumber|documentService|   dueDate|fiscalYear|postingDate|totalPaid|
+-----+-------------+----------------+--------------+---------------+----------+----------+-----------+---------+
| 6048| Customer_197|             USD|    2022-00264|      Insurance|2022-01-24|      2022| 2022-01-06|     1996|
|  517|Customer_6420|             EUR|    2022-00534|        Utility|2022-01-19|      2022| 2022-01-11|      459|
| 9796|Customer_7227|             GBR|    2022-00609|      Insurance|2022-02-09|      2022| 2022-01-12|     NULL|
| 3203|Customer_8294|             CHF|    2022-00651|      Insurance|2022-02-07|      2022| 2022-01-13|     2126|
| 5353| Customer_673|             EUR|    2022-00826|      Insurance|2022-03-05|      2022| 2022-01-17|      151|
| 8309| Customer_575|             USD|    2022-00892|           Rent|2022-02-04|      20